# Parkinson's Disease Detection from Voice Data

## Project Overview

**Objective**: Develop a classification model to predict whether a person has Parkinson's disease using vocal biomarkers extracted from sustained phonations. Early diagnosis may help patients get better care and treatment.

**Dataset**: UCI Parkinson's Dataset

In [1]:
# 1.1 Download the data with wget to raw_data folder

!mkdir -p ../data/raw

!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data -O ../data/raw/parkinsons.data
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.names -O ../data/raw/parkinsons.names

In [2]:
# 2.1 Environment setup
# !conda env create -f env.yaml

In [3]:
# 2.2 Command for activate environment
# !conda activate university-machine-learning-project-parkinsons-voice-v3

In [4]:
# 3.1 Load data/parkinsons.data with correct headers
# Setup: Import libraries and create organized folder structure
import json
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import (
    SelectKBest,
    f_classif,
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    balanced_accuracy_score,
)
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

plt.style.use("default")
sns.set_palette("husl")


# Create organized folder structure
def create_project_structure():
    """Create clean, organized project structure"""
    folders = [
        "../data/raw",
        "../data/processed",
        "../models",
        "../results",
        "../figures",
    ]

    for folder in folders:
        Path(folder).mkdir(parents=True, exist_ok=True)

    print("Project structure created")


create_project_structure()

# Global settings
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("=== PARKINSON'S DISEASE DETECTION PIPELINE ===")
print(f"Random state: {RANDOM_STATE}")

Project structure created
=== PARKINSON'S DISEASE DETECTION PIPELINE ===
Random state: 42


In [5]:
# 3.1-3.4: Load and analyze data
# 3.1-3.4: Load and analyze data
def load_and_analyze_data():
    """Load data and perform basic analysis"""
    print("\n=== LOADING DATA ===")

    # Load data
    df = pd.read_csv("../data/raw/parkinsons.data")

    # Basic info
    print(f"Dataset shape: {df.shape}")
    print(f"Missing values: {df.isnull().sum().sum()}")
    print(f"Duplicates: {df.duplicated().sum()}")

    # Target analysis
    target_counts = df["status"].value_counts()
    print(f"\nTarget distribution:")
    print(f"  Healthy (0): {target_counts[0]} ({target_counts[0]/len(df)*100:.1f}%)")
    print(
        f"  Parkinson's (1): {target_counts[1]} ({target_counts[1]/len(df)*100:.1f}%)"
    )

    # Save target distribution plot
    plt.figure(figsize=(8, 3))
    plt.subplot(1, 2, 1)
    target_counts.plot(kind="bar", color=["lightblue", "lightcoral"])
    plt.title("Target Distribution")
    plt.xticks([0, 1], ["Healthy", "Parkinson's"], rotation=0)

    plt.subplot(1, 2, 2)
    plt.pie(
        target_counts.values,
        labels=["Healthy", "Parkinson's"],
        autopct="%1.1f%%",
        colors=["lightblue", "lightcoral"],
    )
    plt.title("Target Proportions")

    plt.tight_layout()
    plt.savefig("../figures/target_distribution.png", dpi=150, bbox_inches="tight")
    plt.close()

    return df


df = load_and_analyze_data()


=== LOADING DATA ===
Dataset shape: (195, 24)
Missing values: 0
Duplicates: 0

Target distribution:
  Healthy (0): 48 (24.6%)
  Parkinson's (1): 147 (75.4%)


In [6]:
# 3.5-3.8: Statistical analysis
def statistical_analysis(df):
    """Perform statistical analysis and effect size calculations"""
    print("\n=== STATISTICAL ANALYSIS ===")

    feature_cols = [col for col in df.columns if col not in ["name", "status"]]
    healthy = df[df["status"] == 0]
    parkinsons = df[df["status"] == 1]

    # Mann-Whitney U tests
    results = []
    for feature in feature_cols:
        stat, p_val = mannwhitneyu(
            healthy[feature], parkinsons[feature], alternative="two-sided"
        )

        # Cohen's d
        mean1, std1 = healthy[feature].mean(), healthy[feature].std()
        mean2, std2 = parkinsons[feature].mean(), parkinsons[feature].std()
        pooled_std = np.sqrt(
            ((len(healthy) - 1) * std1**2 + (len(parkinsons) - 1) * std2**2)
            / (len(healthy) + len(parkinsons) - 2)
        )
        cohens_d = (mean2 - mean1) / pooled_std

        results.append(
            {
                "feature": feature,
                "p_value": p_val,
                "cohens_d": cohens_d,
                "effect_size": (
                    "large"
                    if abs(cohens_d) > 0.8
                    else "medium" if abs(cohens_d) > 0.5 else "small"
                ),
            }
        )

    results_df = pd.DataFrame(results).sort_values("p_value")

    # FDR correction
    _, p_fdr, _, _ = multipletests(results_df["p_value"], method="fdr_bh")
    results_df["p_fdr"] = p_fdr

    # Show top significant features
    significant = results_df[results_df["p_fdr"] < 0.05].head(10)
    print(f"Top 10 significant features (FDR < 0.05):")
    for _, row in significant.iterrows():
        print(
            f"  {row['feature']}: p={row['p_value']:.2e}, Cohen's d={row['cohens_d']:.3f} ({row['effect_size']})"
        )

    return results_df


stats_results = statistical_analysis(df)


=== STATISTICAL ANALYSIS ===
Top 10 significant features (FDR < 0.05):
  PPE: p=1.59e-16, Cohen's d=1.447 (large)
  spread1: p=1.59e-16, Cohen's d=1.581 (large)
  MDVP:APQ: p=1.27e-11, Cohen's d=0.903 (large)
  spread2: p=7.16e-11, Cohen's d=1.180 (large)
  MDVP:Jitter(Abs): p=1.28e-09, Cohen's d=0.831 (large)
  MDVP:PPQ: p=2.40e-09, Cohen's d=0.696 (medium)
  MDVP:Shimmer(dB): p=3.14e-09, Cohen's d=0.865 (large)
  MDVP:Shimmer: p=4.22e-09, Cohen's d=0.912 (large)
  MDVP:Jitter(%): p=7.90e-09, Cohen's d=0.669 (medium)
  Jitter:DDP: p=8.18e-09, Cohen's d=0.639 (medium)


In [7]:
# 3.8: Statistical power analysis
def statistical_power_analysis(stats_results, df):
    """Conduct post-hoc statistical power analysis using observed Cohen's d"""
    print("\n=== STATISTICAL POWER ANALYSIS ===")

    # Calculate statistical power for observed effect sizes
    # Using conventional alpha = 0.05 and sample sizes
    healthy_n = len(df[df["status"] == 0])
    parkinsons_n = len(df[df["status"] == 1])
    total_n = len(df)

    print(
        f"Sample sizes: Healthy = {healthy_n}, Parkinson's = {parkinsons_n}, Total = {total_n}"
    )

    # Calculate statistical power for Cohen's d effect sizes
    # Power = 1 - β (where β is Type II error rate)
    # Using approximation: Power ≈ Φ(|d|√(n/2) - z_α/2) where Φ is normal CDF

    z_alpha_2 = 1.96  # Critical value for α/2 = 0.025

    # Calculate effective sample size (harmonic mean for unequal groups)
    n_effective = 2 * healthy_n * parkinsons_n / (healthy_n + parkinsons_n)

    power_results = []
    for _, row in stats_results.iterrows():
        feature = row["feature"]
        cohens_d = abs(row["cohens_d"])

        # Power calculation using normal approximation
        # Power ≈ Φ(|d|√(n_eff/2) - z_α/2)
        z_score = cohens_d * np.sqrt(n_effective / 2) - z_alpha_2

        # Use normal CDF to get power
        from scipy.stats import norm

        power = norm.cdf(z_score)

        # Classify power levels
        if power >= 0.8:
            power_level = "High"
        elif power >= 0.5:
            power_level = "Medium"
        else:
            power_level = "Low"

        power_results.append(
            {
                "feature": feature,
                "cohens_d": cohens_d,
                "power": power,
                "power_level": power_level,
            }
        )

    power_df = pd.DataFrame(power_results).sort_values("power", ascending=False)

    # Display top features by power
    print(f"\nTop 10 features by statistical power:")
    for _, row in power_df.head(10).iterrows():
        print(
            f"  {row['feature']}: Power={row['power']:.3f} ({row['power_level']}), Cohen's d={row['cohens_d']:.3f}"
        )

    # Summary statistics
    high_power = len(power_df[power_df["power"] >= 0.8])
    medium_power = len(power_df[(power_df["power"] >= 0.5) & (power_df["power"] < 0.8)])
    low_power = len(power_df[power_df["power"] < 0.5])

    print(f"\nPower distribution:")
    print(f"  High power (≥0.8): {high_power} features")
    print(f"  Medium power (0.5-0.8): {medium_power} features")
    print(f"  Low power (<0.5): {low_power} features")

    # Save power analysis results
    power_df.to_csv("../results/statistical_power_analysis.csv", index=False)
    print("✓ Power analysis results saved")

    return power_df


power_results = statistical_power_analysis(stats_results, df)


=== STATISTICAL POWER ANALYSIS ===
Sample sizes: Healthy = 48, Parkinson's = 147, Total = 195

Top 10 features by statistical power:
  spread1: Power=1.000 (High), Cohen's d=1.581
  PPE: Power=1.000 (High), Cohen's d=1.447
  spread2: Power=1.000 (High), Cohen's d=1.180
  MDVP:Fo(Hz): Power=1.000 (High), Cohen's d=0.959
  MDVP:Flo(Hz): Power=1.000 (High), Cohen's d=0.949
  MDVP:Shimmer: Power=1.000 (High), Cohen's d=0.912
  MDVP:APQ: Power=1.000 (High), Cohen's d=0.903
  HNR: Power=1.000 (High), Cohen's d=0.895
  Shimmer:APQ5: Power=0.999 (High), Cohen's d=0.866
  MDVP:Shimmer(dB): Power=0.999 (High), Cohen's d=0.865

Power distribution:
  High power (≥0.8): 20 features
  Medium power (0.5-0.8): 2 features
  Low power (<0.5): 0 features
✓ Power analysis results saved


In [8]:
# 3.9: Key feature visualization
def visualize_key_features(df, stats_results):
    """Visualize distributions of top discriminative features"""
    print("\n=== VISUALIZING KEY FEATURES ===")

    # Select top 6 features by effect size
    top_features = stats_results.nlargest(6, "cohens_d")["feature"].tolist()

    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.ravel()

    for i, feature in enumerate(top_features):
        ax = axes[i]

        healthy_data = df[df["status"] == 0][feature]
        parkinsons_data = df[df["status"] == 1][feature]

        ax.hist(
            healthy_data,
            bins=20,
            alpha=0.6,
            label="Healthy",
            color="lightblue",
            density=True,
        )
        ax.hist(
            parkinsons_data,
            bins=20,
            alpha=0.6,
            label="Parkinson's",
            color="lightcoral",
            density=True,
        )

        ax.set_title(f"{feature}")
        ax.set_xlabel("Value")
        ax.set_ylabel("Density")
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.suptitle("Key Discriminative Features", fontsize=14)
    plt.tight_layout()
    plt.savefig(
        "../figures/key_features_distributions.png", dpi=150, bbox_inches="tight"
    )
    plt.close()

    print(f"✓ Saved distributions for {len(top_features)} key features")


visualize_key_features(df, stats_results)


=== VISUALIZING KEY FEATURES ===
✓ Saved distributions for 6 key features


In [9]:
# 3.10: Correlation analysis
def correlation_analysis(df):
    """Analyze feature correlations"""
    print("\n=== CORRELATION ANALYSIS ===")

    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    corr_matrix = df[numeric_cols].corr()

    # Plot correlation heatmap
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(
        corr_matrix,
        mask=mask,
        center=0,
        cmap="RdBu_r",
        square=True,
        cbar_kws={"shrink": 0.8},
    )
    plt.title("Feature Correlation Matrix")
    plt.tight_layout()
    plt.savefig("../figures/correlation_heatmap.png", dpi=150, bbox_inches="tight")
    plt.close()

    # Target correlations
    target_corr = (
        corr_matrix["status"].drop("status").sort_values(key=abs, ascending=False)
    )
    print(f"Top 15 target correlations:")
    for feature, corr in target_corr.head(15).items():
        print(f"  {feature}: {corr:.3f}")

    return corr_matrix


corr_matrix = correlation_analysis(df)


=== CORRELATION ANALYSIS ===
Top 15 target correlations:
  spread1: 0.565
  PPE: 0.531
  spread2: 0.455
  MDVP:Fo(Hz): -0.384
  MDVP:Flo(Hz): -0.380
  MDVP:Shimmer: 0.367
  MDVP:APQ: 0.364
  HNR: -0.362
  Shimmer:APQ5: 0.351
  MDVP:Shimmer(dB): 0.351
  Shimmer:APQ3: 0.348
  Shimmer:DDA: 0.348
  D2: 0.340
  MDVP:Jitter(Abs): 0.339
  RPDE: 0.309


In [10]:
# 4.1-4.6: Feature engineering
def engineer_features(df):
    """Create engineered features"""
    print("\n=== FEATURE ENGINEERING ===")

    df_eng = df.copy()

    # Create composite features
    df_eng["jitter_stability_ratio"] = df_eng["MDVP:Jitter(%)"] / (
        df_eng["MDVP:Jitter(Abs)"] + 1e-8
    )
    df_eng["shimmer_composite"] = (
        df_eng["MDVP:Shimmer"] + df_eng["Shimmer:APQ3"] + df_eng["Shimmer:APQ5"]
    ) / 3
    df_eng["voice_quality_index"] = df_eng["HNR"] / (df_eng["NHR"] + 1)
    df_eng["frequency_range"] = df_eng["MDVP:Fhi(Hz)"] - df_eng["MDVP:Flo(Hz)"]
    df_eng["frequency_cv"] = df_eng["frequency_range"] / df_eng["MDVP:Fo(Hz)"]

    new_features = [
        "jitter_stability_ratio",
        "shimmer_composite",
        "voice_quality_index",
        "frequency_range",
        "frequency_cv",
    ]

    print(f"✓ Created {len(new_features)} engineered features")
    
    # Print the new features
    print(f"New features: {new_features}")

    # Save engineered dataset
    df_eng.to_csv("../data/processed/parkinsons_engineered.csv", index=False)
    
    # Calculate the correlation of the new features with the target
    corr_with_target = df_eng[new_features].corrwith(df_eng["status"])
    print(f"Correlation of new features with the target:\n{corr_with_target}")

    return df_eng


df_engineered = engineer_features(df)


=== FEATURE ENGINEERING ===
✓ Created 5 engineered features
New features: ['jitter_stability_ratio', 'shimmer_composite', 'voice_quality_index', 'frequency_range', 'frequency_cv']
Correlation of new features with the target:
jitter_stability_ratio   -0.435975
shimmer_composite         0.360146
voice_quality_index      -0.358165
frequency_range           0.013754
frequency_cv              0.033349
dtype: float64


In [11]:
# 5.2: LOSO CV split by name and status stratification
def prepare_data_loso(df):
    """Prepare data with Leave-One-Subject-Out (LOSO) cross-validation setup"""
    print("\n=== DATA PREPARATION WITH LOSO CV ===")

    # Prepare features and target
    feature_cols = [col for col in df.columns if col not in ["name", "status"]]
    X = df[feature_cols]
    y = df["status"]
    names = df["name"]

    print(f"Total subjects: {len(df['name'].unique())}")
    print(f"Features: {len(feature_cols)}")

    # For demonstration, we'll use stratified split but keep name information
    # In practice, LOSO would iterate through each subject as test set

    # Create subject-aware stratified split
    # Group by subject name to ensure no subject appears in both train and test
    unique_subjects = df["name"].unique()
    np.random.seed(RANDOM_STATE)
    np.random.shuffle(unique_subjects)

    # Calculate split point to maintain roughly 70-30 split while respecting subjects
    n_test_subjects = max(1, int(len(unique_subjects) * 0.3))
    test_subjects = unique_subjects[:n_test_subjects]
    train_subjects = unique_subjects[n_test_subjects:]

    # Create train/test splits based on subject names
    train_mask = df["name"].isin(train_subjects)
    test_mask = df["name"].isin(test_subjects)

    X_train = X[train_mask]
    X_test = X[test_mask]
    y_train = y[train_mask]
    y_test = y[test_mask]

    print(f"Train subjects: {len(train_subjects)} ({len(X_train)} samples)")
    print(f"Test subjects: {len(test_subjects)} ({len(X_test)} samples)")
    print(f"Train class distribution: {pd.Series(y_train).value_counts().to_dict()}")
    print(f"Test class distribution: {pd.Series(y_test).value_counts().to_dict()}")

    # Verify no subject overlap
    train_subject_set = set(df[train_mask]["name"].unique())
    test_subject_set = set(df[test_mask]["name"].unique())
    overlap = train_subject_set.intersection(test_subject_set)

    if len(overlap) == 0:
        print("✓ No subject overlap between train and test sets")
    else:
        print(f"⚠ Warning: {len(overlap)} subjects found in both sets: {overlap}")

    return X_train, X_test, y_train, y_test, train_subjects, test_subjects


X_train, X_test, y_train, y_test, train_subjects, test_subjects = prepare_data_loso(
    df_engineered
)


=== DATA PREPARATION WITH LOSO CV ===
Total subjects: 195
Features: 27
Train subjects: 137 (137 samples)
Test subjects: 58 (58 samples)
Train class distribution: {1: 103, 0: 34}
Test class distribution: {1: 44, 0: 14}
✓ No subject overlap between train and test sets


In [12]:
# 5.3: Scale features
def scale_features(X_train, X_test):
    """Scale features using RobustScaler"""
    print("\n=== FEATURE SCALING ===")

    scaler = RobustScaler()
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index
    )
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test), columns=X_test.columns, index=X_test.index
    )

    print("✓ Features scaled with RobustScaler")
    return X_train_scaled, X_test_scaled, scaler



X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)


=== FEATURE SCALING ===
✓ Features scaled with RobustScaler


In [13]:
# 5.4: SMOTE and feature selection
def apply_smote_and_feature_selection(X_train, y_train):
    """Apply SMOTE and feature selection"""
    print("\n=== SMOTE & FEATURE SELECTION ===")

    # SMOTE
    smote = SMOTE(random_state=RANDOM_STATE, k_neighbors=3)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    print(f"After SMOTE: {pd.Series(y_train_smote).value_counts().to_dict()}")

    # Feature selection using multiple methods
    feature_selectors = {}

    # Statistical selection
    selector_stat = SelectKBest(score_func=f_classif, k=15)
    selector_stat.fit(X_train, y_train)
    selected_stat = X_train.columns[selector_stat.get_support()].tolist()
    feature_selectors["statistical"] = selected_stat

    # RF importance
    rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
    rf.fit(X_train, y_train)
    importances = pd.Series(rf.feature_importances_, index=X_train.columns)
    selected_rf = importances.nlargest(15).index.tolist()
    feature_selectors["random_forest"] = selected_rf

    # Consensus features (appearing in both methods)
    consensus_features = list(set(selected_stat) & set(selected_rf))
    if len(consensus_features) < 10:
        # If too few consensus features, take top from union
        all_selected = list(set(selected_stat + selected_rf))
        consensus_features = all_selected[:15]

    print(f"✓ Selected {len(consensus_features)} consensus features")
    print(f"Consensus features: {consensus_features}")
    
    # Calculate the correlation of the new features with the target
    corr_with_target = X_train_smote[consensus_features].corrwith(y_train_smote)
    print(f"Correlation of consensus features with the target:\n{corr_with_target}")

    return X_train_smote, y_train_smote, consensus_features, feature_selectors


X_train_smote, y_train_smote, consensus_features, feature_selectors = (
    apply_smote_and_feature_selection(X_train_scaled, y_train)
)


=== SMOTE & FEATURE SELECTION ===
After SMOTE: {1: 103, 0: 103}
✓ Selected 15 consensus features
Consensus features: ['shimmer_composite', 'frequency_range', 'voice_quality_index', 'MDVP:Shimmer(dB)', 'NHR', 'MDVP:RAP', 'MDVP:Shimmer', 'MDVP:PPQ', 'MDVP:Fhi(Hz)', 'MDVP:APQ', 'PPE', 'MDVP:Fo(Hz)', 'Shimmer:APQ3', 'spread1', 'Shimmer:DDA']
Correlation of consensus features with the target:
shimmer_composite      0.488288
frequency_range        0.054313
voice_quality_index   -0.482157
MDVP:Shimmer(dB)       0.478504
NHR                    0.300172
MDVP:RAP               0.376226
MDVP:Shimmer           0.494679
MDVP:PPQ               0.402908
MDVP:Fhi(Hz)          -0.193859
MDVP:APQ               0.483743
PPE                    0.628846
MDVP:Fo(Hz)           -0.387142
Shimmer:APQ3           0.476398
spread1                0.646348
Shimmer:DDA            0.476367
dtype: float64


In [14]:
# 5.5: Create dataset variants
def create_dataset_variants(
    X_train, X_test, X_train_smote, y_train, y_train_smote, y_test, consensus_features
):
    """Create different dataset variants for model comparison"""
    print("\n=== CREATING DATASET VARIANTS ===")

    variants = {
        "baseline": {
            "X_train": X_train,
            "X_test": X_test,
            "y_train": y_train,
            "y_test": y_test,
        },
        "smote": {
            "X_train": pd.DataFrame(X_train_smote, columns=X_train.columns),
            "X_test": X_test,
            "y_train": pd.Series(y_train_smote),
            "y_test": y_test,
        },
        "feature_selected": {
            "X_train": X_train[consensus_features],
            "X_test": X_test[consensus_features],
            "y_train": y_train,
            "y_test": y_test,
        },
        "smote_feature": {
            "X_train": pd.DataFrame(X_train_smote, columns=X_train.columns)[
                consensus_features
            ],
            "X_test": X_test[consensus_features],
            "y_train": pd.Series(y_train_smote),
            "y_test": y_test,
        },
    }

    # PCA variant
    pca = PCA(n_components=0.95, random_state=RANDOM_STATE)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    variants["pca"] = {
        "X_train": pd.DataFrame(
            X_train_pca, columns=[f"PC{i+1}" for i in range(X_train_pca.shape[1])]
        ),
        "X_test": pd.DataFrame(
            X_test_pca, columns=[f"PC{i+1}" for i in range(X_test_pca.shape[1])]
        ),
        "y_train": y_train,
        "y_test": y_test,
    }

    print(f"✓ Created {len(variants)} dataset variants")
    
    # Print the name of the dataset variants
    print(f"Dataset variants: {list(variants.keys())}")

    # Save variants
    for name, data in variants.items():
        data["X_train"].to_csv(f"../data/processed/X_train_{name}.csv", index=False)
        data["X_test"].to_csv(f"../data/processed/X_test_{name}.csv", index=False)
        pd.Series(data["y_train"]).to_csv(
            f"../data/processed/y_train_{name}.csv", index=False, header=["status"]
        )
        pd.Series(data["y_test"]).to_csv(
            f"../data/processed/y_test_{name}.csv", index=False, header=["status"]
        )

    return variants, pca


variants, pca = create_dataset_variants(
    X_train_scaled,
    X_test_scaled,
    X_train_smote,
    y_train,
    y_train_smote,
    y_test,
    consensus_features,
)


=== CREATING DATASET VARIANTS ===
✓ Created 5 dataset variants
Dataset variants: ['baseline', 'smote', 'feature_selected', 'smote_feature', 'pca']


In [15]:
# 6.1-6.5: Model training
def train_models(variants):
    """Train multiple models on different dataset variants"""
    print("\n=== MODEL TRAINING ===")

    # Define algorithms
    algorithms = {
        "RandomForest": RandomForestClassifier(
            n_estimators=100,
            random_state=RANDOM_STATE,
            class_weight="balanced",
            n_jobs=5,
        ),
        "SVM": SVC(
            random_state=RANDOM_STATE, class_weight="balanced", probability=True
        ),
        "LogisticRegression": LogisticRegression(
            random_state=RANDOM_STATE, class_weight="balanced", max_iter=1000
        ),
        "KNN": KNeighborsClassifier(n_neighbors=5),
        "DecisionTree": DecisionTreeClassifier(
            random_state=RANDOM_STATE, class_weight="balanced"
        ),
    }

    algorithms["XGBoost"] = XGBClassifier(
        random_state=RANDOM_STATE, eval_metric="logloss", verbosity=0
    )

    # Hyperparameter grids (simplified for efficiency)
    param_grids = {
        "RandomForest": {"n_estimators": [100, 200], "max_depth": [10, None]},
        "SVM": {"C": [1, 10], "kernel": ["rbf", "linear"]},
        "LogisticRegression": {"C": [0.1, 1, 10]},
        "KNN": {"n_neighbors": [3, 5, 7]},
        "DecisionTree": {"max_depth": [5, 10, None]},
    }

    param_grids["XGBoost"] = {"n_estimators": [100, 200], "max_depth": [3, 5]}

    results = []
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    for variant_name, data in variants.items():
        print(f"\nTraining on {variant_name} dataset...")

        X_train_var = data["X_train"]
        y_train_var = data["y_train"]

        for algo_name, model in algorithms.items():
            try:
                # Grid search
                grid_search = GridSearchCV(
                    model,
                    param_grids.get(algo_name, {}),
                    cv=cv,
                    scoring="f1",
                    n_jobs=5,
                    verbose=0,
                )

                grid_search.fit(X_train_var, y_train_var)

                # Test evaluation
                X_test_var = data["X_test"]
                y_test_var = data["y_test"]
                y_pred = grid_search.predict(X_test_var)

                # Metrics
                metrics = {
                    "algorithm": algo_name,
                    "dataset": variant_name,
                    "cv_f1": grid_search.best_score_,
                    "test_f1": f1_score(y_test_var, y_pred),
                    "test_accuracy": accuracy_score(y_test_var, y_pred),
                    "test_precision": precision_score(
                        y_test_var, y_pred, zero_division=0
                    ),
                    "test_recall": recall_score(y_test_var, y_pred),
                    "best_params": grid_search.best_params_,
                    "model": grid_search.best_estimator_,
                }

                results.append(metrics)
                print(f"  {algo_name}: F1={metrics['test_f1']:.3f}")

            except Exception as e:
                print(f"  {algo_name}: Failed - {str(e)[:50]}")

    return results


training_results = train_models(variants)


=== MODEL TRAINING ===

Training on baseline dataset...
  RandomForest: F1=0.946
  SVM: F1=0.966
  LogisticRegression: F1=0.907
  KNN: F1=0.944
  DecisionTree: F1=0.909
  XGBoost: F1=0.957

Training on smote dataset...
  RandomForest: F1=0.956
  SVM: F1=0.966
  LogisticRegression: F1=0.921
  KNN: F1=0.929
  DecisionTree: F1=0.851
  XGBoost: F1=0.956

Training on feature_selected dataset...
  RandomForest: F1=0.945
  SVM: F1=0.955
  LogisticRegression: F1=0.921
  KNN: F1=0.945
  DecisionTree: F1=0.944
  XGBoost: F1=0.933

Training on smote_feature dataset...
  RandomForest: F1=0.956
  SVM: F1=0.932
  LogisticRegression: F1=0.933
  KNN: F1=0.907
  DecisionTree: F1=0.884
  XGBoost: F1=0.944

Training on pca dataset...
  RandomForest: F1=0.946
  SVM: F1=0.955
  LogisticRegression: F1=0.899
  KNN: F1=0.933
  DecisionTree: F1=0.933
  XGBoost: F1=0.936


In [16]:
# 6.6-6.7: Select top models
def select_top_models(results, n_top=5):
    """Select top performing models"""
    print(f"\n=== SELECTING TOP {n_top} MODELS ===")

    # Sort by test F1 score
    sorted_results = sorted(results, key=lambda x: x["test_f1"], reverse=True)
    top_models = sorted_results[:n_top]

    print("Top models:")
    for i, model in enumerate(top_models, 1):
        print(
            f"{i}. {model['algorithm']} ({model['dataset']}): F1={model['test_f1']:.3f}"
        )

    # Save top models
    for i, model in enumerate(top_models):
        model_name = f"model_{i+1}_{model['algorithm']}_{model['dataset']}"
        joblib.dump(model["model"], f"../models/{model_name}.pkl")

    return top_models


top_models = select_top_models(training_results)


=== SELECTING TOP 5 MODELS ===
Top models:
1. SVM (baseline): F1=0.966
2. SVM (smote): F1=0.966
3. XGBoost (baseline): F1=0.957
4. RandomForest (smote): F1=0.956
5. XGBoost (smote): F1=0.956


In [17]:
# 7.1-7.3: Evaluate top models
def evaluate_top_models(top_models, variants):
    """Comprehensive evaluation of top models"""
    print("\n=== MODEL EVALUATION ===")

    evaluation_results = []

    for i, model_info in enumerate(top_models):
        algorithm = model_info["algorithm"]
        dataset = model_info["dataset"]
        model = model_info["model"]

        # Get test data
        data = variants[dataset]
        X_test = data["X_test"]
        y_test = data["y_test"]

        # Predictions
        y_pred = model.predict(X_test)
        y_proba = (
            model.predict_proba(X_test)[:, 1]
            if hasattr(model, "predict_proba")
            else None
        )

        # Comprehensive metrics
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        metrics = {
            "rank": i + 1,
            "algorithm": algorithm,
            "dataset": dataset,
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, zero_division=0),
            "recall": recall_score(y_test, y_pred),  # sensitivity
            "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
            "f1": f1_score(y_test, y_pred),
            "balanced_accuracy": balanced_accuracy_score(y_test, y_pred),
            "roc_auc": roc_auc_score(y_test, y_proba) if y_proba is not None else None,
            "tn": tn,
            "fp": fp,
            "fn": fn,
            "tp": tp,
        }

        evaluation_results.append(metrics)

        # Save confusion matrix
        cm = np.array([[tn, fp], [fn, tp]])
        plt.figure(figsize=(4, 3))
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=["Healthy", "Parkinson"],
            yticklabels=["Healthy", "Parkinson"],
        )
        plt.title(f"{algorithm} ({dataset})")
        plt.ylabel("Actual")
        plt.xlabel("Predicted")
        plt.tight_layout()
        plt.savefig(f"../figures/confusion_matrix_{algorithm}_{dataset}.png", dpi=150)
        plt.close()

    # Save evaluation results
    eval_df = pd.DataFrame(evaluation_results)
    eval_df.to_csv("../results/model_evaluation.csv", index=False)
    
    # Print evaluation results
    print(eval_df)

    return evaluation_results


eval_results = evaluate_top_models(top_models, variants)


=== MODEL EVALUATION ===
   rank     algorithm   dataset  accuracy  precision    recall  specificity  \
0     1           SVM  baseline  0.948276   0.955556  0.977273     0.857143   
1     2           SVM     smote  0.948276   0.955556  0.977273     0.857143   
2     3       XGBoost  baseline  0.931034   0.916667  1.000000     0.714286   
3     4  RandomForest     smote  0.931034   0.934783  0.977273     0.785714   
4     5       XGBoost     smote  0.931034   0.934783  0.977273     0.785714   

         f1  balanced_accuracy   roc_auc  tn  fp  fn  tp  
0  0.966292           0.917208  0.964286  12   2   1  43  
1  0.966292           0.917208  0.970779  12   2   1  43  
2  0.956522           0.857143  0.957792  10   4   0  44  
3  0.955556           0.881494  0.972403  11   3   1  43  
4  0.955556           0.881494  0.975649  11   3   1  43  


In [18]:
# 7.4-7.5: Final model selection and saving
def select_final_model(eval_results, top_models):
    """Select and save the final best model"""
    print("\n=== FINAL MODEL SELECTION ===")

    # Best model by F1 score
    best_model_info = max(eval_results, key=lambda x: x["f1"])
    best_idx = best_model_info["rank"] - 1

    print(f"Best model: {best_model_info['algorithm']} ({best_model_info['dataset']})")
    print(
        f"F1: {best_model_info['f1']:.3f}, Accuracy: {best_model_info['accuracy']:.3f}"
    )
    print(
        f"Sensitivity: {best_model_info['recall']:.3f}, Specificity: {best_model_info['specificity']:.3f}"
    )

    # Save final model
    final_model = top_models[best_idx]["model"]
    joblib.dump(final_model, "../models/final_model.pkl")

    return final_model, best_model_info


final_model, best_model_info = select_final_model(eval_results, top_models)


=== FINAL MODEL SELECTION ===
Best model: SVM (baseline)
F1: 0.966, Accuracy: 0.948
Sensitivity: 0.977, Specificity: 0.857


In [19]:
# 7.6-7.8: Model explainability with comprehensive SHAP analysis
def model_explainability(final_model, best_model_info, variants):
    """Generate comprehensive model explanations using SHAP and feature importance"""
    print("\n=== MODEL EXPLAINABILITY ===")

    dataset = best_model_info["dataset"]
    X_test = variants[dataset]["X_test"]
    X_train = variants[dataset]["X_train"]
    y_test = variants[dataset]["y_test"]

    # 1. Traditional Feature Importance (for tree-based models)
    if hasattr(final_model, "feature_importances_"):
        print("\n--- Traditional Feature Importance ---")
        importances = pd.Series(
            final_model.feature_importances_, index=X_test.columns
        ).sort_values(ascending=False)

        print("Top 10 feature importances:")
        for feature, importance in importances.head(10).items():
            print(f"  {feature}: {importance:.3f}")

        # Plot feature importance
        plt.figure(figsize=(10, 6))
        importances.head(15).plot(kind="barh")
        plt.title(f'Traditional Feature Importance - {best_model_info["algorithm"]}')
        plt.xlabel("Importance")
        plt.tight_layout()
        plt.savefig(
            "../figures/feature_importance_traditional.png",
            dpi=150,
            bbox_inches="tight",
        )
        plt.close()
        print("✓ Traditional feature importance saved")

    # 2. SHAP Analysis
    try:
        import shap

        print("\n--- SHAP Analysis ---")

        # Initialize SHAP explainer based on model type
        if hasattr(final_model, "feature_importances_") and hasattr(
            final_model, "predict_proba"
        ):
            # Tree-based models
            print("Using TreeExplainer for tree-based model...")
            explainer = shap.TreeExplainer(final_model)
            X_explain = X_test.sample(
                min(100, len(X_test)), random_state=42
            )  # Sample for efficiency
            shap_values = explainer.shap_values(X_explain)

            # For binary classification, use positive class SHAP values
            if isinstance(shap_values, list) and len(shap_values) == 2:
                shap_values_pos = shap_values[1]  # Positive class (Parkinson's)
                expected_value = explainer.expected_value[1]
            else:
                shap_values_pos = shap_values
                expected_value = explainer.expected_value

        else:
            # Linear models or other models
            print("Using LinearExplainer/KernelExplainer for non-tree model...")
            if hasattr(final_model, "coef_"):
                # Linear models
                explainer = shap.LinearExplainer(
                    final_model, X_train.sample(min(100, len(X_train)), random_state=42)
                )
            else:
                # General kernel explainer (slower)
                background = shap.sample(X_train, min(50, len(X_train)))
                if hasattr(final_model, "predict_proba"):
                    explainer = shap.KernelExplainer(
                        final_model.predict_proba, background
                    )
                else:
                    explainer = shap.KernelExplainer(final_model.predict, background)

            X_explain = X_test.sample(
                min(50, len(X_test)), random_state=42
            )  # Smaller sample for kernel
            shap_values = explainer.shap_values(X_explain)

            # Handle binary classification
            if isinstance(shap_values, list) and len(shap_values) == 2:
                shap_values_pos = shap_values[1]
                expected_value = (
                    explainer.expected_value[1]
                    if hasattr(explainer.expected_value, "__len__")
                    else explainer.expected_value
                )
            elif len(shap_values.shape) == 3 and shap_values.shape[2] == 2:
                # Handle 3D array from some explainers (shape: n_samples, n_features, n_classes)
                shap_values_pos = shap_values[:, :, 1]  # Use positive class
                expected_value = (
                    explainer.expected_value[1]
                    if hasattr(explainer.expected_value, "__len__")
                    else explainer.expected_value
                )
            else:
                shap_values_pos = shap_values
                expected_value = explainer.expected_value

        # 3. Generate SHAP Visualizations
        print("Generating SHAP visualizations...")

        # Convert to numpy arrays and validate data
        X_explain_values = X_explain.values
        feature_names = X_explain.columns.tolist()

        # Debug information
        print(f"SHAP values shape: {shap_values_pos.shape}")
        print(f"X_explain shape: {X_explain_values.shape}")
        print(f"Feature names length: {len(feature_names)}")
        print(f"Expected value type: {type(expected_value)}")

        # Ensure data is clean (no NaN/inf values)
        if np.any(np.isnan(shap_values_pos)) or np.any(np.isinf(shap_values_pos)):
            print("Warning: NaN or inf values found in SHAP values, cleaning...")
            shap_values_pos = np.nan_to_num(
                shap_values_pos, nan=0.0, posinf=0.0, neginf=0.0
            )

        if np.any(np.isnan(X_explain_values)) or np.any(np.isinf(X_explain_values)):
            print("Warning: NaN or inf values found in X_explain, cleaning...")
            X_explain_values = np.nan_to_num(
                X_explain_values, nan=0.0, posinf=0.0, neginf=0.0
            )

        # Try summary plot with better error handling
        try:
            plt.figure(figsize=(10, 8))
            # Use a simpler approach for summary plot
            shap.summary_plot(
                shap_values_pos,
                X_explain_values,
                feature_names=feature_names,
                plot_type="bar",
                show=False,
                max_display=15,
            )
            plt.title("SHAP Feature Importance (Mean |SHAP value|)")
            plt.tight_layout()
            plt.savefig("../figures/shap_summary_bar.png", dpi=150, bbox_inches="tight")
            plt.close()
            print("✓ Bar plot saved successfully")
        except Exception as e:
            print(f"⚠ Bar plot failed: {str(e)[:100]}")
            plt.close()

            # Fallback: Create manual bar plot
            try:
                feature_importance_shap = np.abs(shap_values_pos).mean(axis=0)
                top_indices = np.argsort(feature_importance_shap)[::-1][:15]

                plt.figure(figsize=(10, 8))
                y_pos = np.arange(len(top_indices))
                plt.barh(y_pos, feature_importance_shap[top_indices])
                plt.yticks(y_pos, [feature_names[i] for i in top_indices])
                plt.xlabel("Mean |SHAP value|")
                plt.title("SHAP Feature Importance (Manual Plot)")
                plt.gca().invert_yaxis()
                plt.tight_layout()
                plt.savefig(
                    "../figures/shap_summary_bar.png", dpi=150, bbox_inches="tight"
                )
                plt.close()
                print("✓ Manual bar plot saved successfully")
            except Exception as e2:
                print(f"⚠ Manual bar plot also failed: {str(e2)[:100]}")
                plt.close()

        # Try detailed summary plot
        try:
            plt.figure(figsize=(10, 8))
            shap.summary_plot(
                shap_values_pos,
                X_explain_values,
                feature_names=feature_names,
                show=False,
                max_display=15,
            )
            plt.title("SHAP Summary Plot (Feature Impact)")
            plt.tight_layout()
            plt.savefig(
                "../figures/shap_summary_detailed.png", dpi=150, bbox_inches="tight"
            )
            plt.close()
            print("✓ Detailed plot saved successfully")
        except Exception as e:
            print(f"⚠ Detailed plot failed: {str(e)[:100]}")
            plt.close()

        # 4. SHAP Feature Ranking
        print("Computing SHAP feature importance...")
        print(f"shap_values_pos shape after processing: {shap_values_pos.shape}")
        feature_importance_shap = np.abs(shap_values_pos).mean(axis=0)
        print(f"feature_importance_shap shape: {feature_importance_shap.shape}")
        print(f"feature_names length: {len(feature_names)}")

        shap_importance_df = pd.DataFrame(
            {"feature": feature_names, "shap_importance": feature_importance_shap}
        ).sort_values("shap_importance", ascending=False)

        print("\nTop 10 features by SHAP importance:")
        for _, row in shap_importance_df.head(10).iterrows():
            print(f"  {row['feature']}: {row['shap_importance']:.3f}")

        # 5. Local Explanations (individual predictions)
        print("\n--- Local SHAP Explanations ---")
        print("Starting local explanations...")

        # Find interesting cases: one correctly classified Parkinson's, one correctly classified Healthy
        y_pred_explain = final_model.predict(X_explain)
        y_true_explain = y_test.loc[X_explain.index]

        # Correctly classified Parkinson's case
        parkinson_correct = X_explain.index[
            (y_true_explain == 1) & (y_pred_explain == 1)
        ]
        if len(parkinson_correct) > 0:
            try:
                idx_parkinson = parkinson_correct[0]
                local_idx = list(X_explain.index).index(idx_parkinson)

                plt.figure(figsize=(10, 6))
                shap.waterfall_plot(
                    shap.Explanation(
                        values=shap_values_pos[local_idx],
                        base_values=expected_value,
                        data=X_explain_values[local_idx],
                        feature_names=feature_names,
                    ),
                    max_display=10,
                    show=False,
                )
                plt.title("SHAP Explanation - Correctly Classified Parkinson's Case")
                plt.tight_layout()
                plt.savefig(
                    "../figures/shap_local_parkinson.png", dpi=150, bbox_inches="tight"
                )
                plt.close()
                print("✓ Parkinson's case waterfall plot saved")
            except Exception as e:
                print(f"⚠ Parkinson's waterfall plot failed: {str(e)[:100]}")
                plt.close()

        # Correctly classified Healthy case
        healthy_correct = X_explain.index[(y_true_explain == 0) & (y_pred_explain == 0)]
        if len(healthy_correct) > 0:
            try:
                idx_healthy = healthy_correct[0]
                local_idx = list(X_explain.index).index(idx_healthy)

                plt.figure(figsize=(10, 6))
                shap.waterfall_plot(
                    shap.Explanation(
                        values=shap_values_pos[local_idx],
                        base_values=expected_value,
                        data=X_explain_values[local_idx],
                        feature_names=feature_names,
                    ),
                    max_display=10,
                    show=False,
                )
                plt.title("SHAP Explanation - Correctly Classified Healthy Case")
                plt.tight_layout()
                plt.savefig(
                    "../figures/shap_local_healthy.png", dpi=150, bbox_inches="tight"
                )
                plt.close()
                print("✓ Healthy case waterfall plot saved")
            except Exception as e:
                print(f"⚠ Healthy waterfall plot failed: {str(e)[:100]}")
                plt.close()

        # 6. Save SHAP values for further analysis
        print("Saving SHAP data...")
        try:
            shap_df = pd.DataFrame(shap_values_pos, columns=feature_names)
            shap_df.to_csv("../results/shap_values.csv")
            shap_importance_df.to_csv(
                "../results/shap_feature_importance.csv", index=False
            )
            print("✓ SHAP data saved successfully")
        except Exception as e:
            print(f"⚠ Error saving SHAP data: {str(e)[:100]}")

        # 7. SHAP vs Traditional Feature Importance Comparison
        if hasattr(final_model, "feature_importances_"):
            # Compare SHAP importance with traditional importance
            traditional_imp = pd.Series(
                final_model.feature_importances_, index=feature_names
            )
            comparison_df = pd.DataFrame(
                {
                    "feature": feature_names,
                    "traditional_importance": traditional_imp.values,
                    "shap_importance": feature_importance_shap,
                }
            ).sort_values("shap_importance", ascending=False)

            plt.figure(figsize=(12, 8))
            x = np.arange(len(comparison_df.head(15)))
            width = 0.35

            plt.bar(
                x - width / 2,
                comparison_df.head(15)["traditional_importance"],
                width,
                label="Traditional Importance",
                alpha=0.8,
            )
            plt.bar(
                x + width / 2,
                comparison_df.head(15)["shap_importance"],
                width,
                label="SHAP Importance",
                alpha=0.8,
            )

            plt.xlabel("Features")
            plt.ylabel("Importance")
            plt.title("Traditional vs SHAP Feature Importance Comparison")
            plt.xticks(x, comparison_df.head(15)["feature"], rotation=45, ha="right")
            plt.legend()
            plt.tight_layout()
            plt.savefig(
                "../figures/importance_comparison.png", dpi=150, bbox_inches="tight"
            )
            plt.close()

            comparison_df.to_csv("../results/importance_comparison.csv", index=False)

        print("✓ SHAP analysis completed successfully!")
        print("✓ SHAP visualizations saved to ../figures/")
        print("✓ SHAP data saved to ../results/")

    except ImportError:
        print(
            "⚠ SHAP not available - install with 'pip install shap' for advanced explanations"
        )
        print("  Run: pip install shap")
    except Exception as e:
        print(f"⚠ SHAP analysis failed: {str(e)[:100]}...")
        print("  Continuing with traditional feature importance only")


model_explainability(final_model, best_model_info, variants)


=== MODEL EXPLAINABILITY ===

--- SHAP Analysis ---
Using LinearExplainer/KernelExplainer for non-tree model...


100%|██████████| 50/50 [00:21<00:00,  2.35it/s]


Generating SHAP visualizations...
SHAP values shape: (50, 27)
X_explain shape: (50, 27)
Feature names length: 27
Expected value type: <class 'numpy.float64'>
✓ Bar plot saved successfully
✓ Detailed plot saved successfully
Computing SHAP feature importance...
shap_values_pos shape after processing: (50, 27)
feature_importance_shap shape: (27,)
feature_names length: 27

Top 10 features by SHAP importance:
  D2: 0.044
  spread2: 0.042
  Jitter:DDP: 0.030
  MDVP:RAP: 0.030
  MDVP:Fhi(Hz): 0.022
  spread1: 0.021
  DFA: 0.021
  frequency_cv: 0.020
  MDVP:Jitter(%): 0.018
  jitter_stability_ratio: 0.017

--- Local SHAP Explanations ---
Starting local explanations...
✓ Parkinson's case waterfall plot saved
✓ Healthy case waterfall plot saved
Saving SHAP data...
✓ SHAP data saved successfully
✓ SHAP analysis completed successfully!
✓ SHAP visualizations saved to ../figures/
✓ SHAP data saved to ../results/


In [20]:
# 7.9: Summary and results
def create_summary():
    """Create final summary"""
    print("\n=== FINAL SUMMARY ===")

    # Training summary
    training_df = pd.DataFrame(
        [
            {
                "algorithm": r["algorithm"],
                "dataset": r["dataset"],
                "test_f1": r["test_f1"],
                "test_accuracy": r["test_accuracy"],
            }
            for r in training_results
        ]
    )

    print(f"Models trained: {len(training_results)}")
    print(f"Best F1 score: {training_df['test_f1'].max():.3f}")
    print(f"Best accuracy: {training_df['test_accuracy'].max():.3f}")

    # Save all results
    results_summary = {
        "training_results": training_results,
        "evaluation_results": eval_results,
        "best_model_info": best_model_info,
        "feature_selectors": feature_selectors,
        "metadata": {
            "random_state": RANDOM_STATE,
            "timestamp": pd.Timestamp.now().isoformat(),
            "n_models_trained": len(training_results),
        },
    }

    with open("../results/summary.json", "w") as f:
        # Convert non-serializable objects
        summary_clean = results_summary.copy()
        summary_clean["training_results"] = [
            {k: v for k, v in r.items() if k != "model"} for r in training_results
        ]
        json.dump(summary_clean, f, indent=2, default=str)

    print("\n✓ Results saved to ../")
    print("✓ Pipeline completed successfully!")


create_summary()

print("\n" + "=" * 50)
print("🎉 PARKINSON'S DISEASE DETECTION PIPELINE COMPLETE")
print("=" * 50)
print("Outputs saved in organized structure:")
print("PARKINSON'S PROJECT")
print("  ├── 📁 notebooks/     (analysis code)")
print("  ├── 📁 data/")
print("  │   ├── 📁 raw/       (original data)")
print("  │   └── 📁 processed/ (preprocessed datasets)")
print("  ├── 📁 models/        (trained models)")
print("  ├── 📁 results/       (evaluation results)")
print("  └── 📁 figures/       (visualizations)")


=== FINAL SUMMARY ===
Models trained: 30
Best F1 score: 0.966
Best accuracy: 0.948

✓ Results saved to ../
✓ Pipeline completed successfully!

🎉 PARKINSON'S DISEASE DETECTION PIPELINE COMPLETE
Outputs saved in organized structure:
PARKINSON'S PROJECT
  ├── 📁 notebooks/     (analysis code)
  ├── 📁 data/
  │   ├── 📁 raw/       (original data)
  │   └── 📁 processed/ (preprocessed datasets)
  ├── 📁 models/        (trained models)
  ├── 📁 results/       (evaluation results)
  └── 📁 figures/       (visualizations)
